In [33]:
import flika_JSON_IO as flikaIO
import feature_calculator as fcalc
import numpy as np
import os
import pandas as pd

In [34]:
working_dir = r'/home/vivek/Documents/Python Programs/piezo1/Piezo1_SVM_alpha/JSON_example_file/'
working_file = 'GB_221_Halo_B2.json'
experiment_name = 'GB_221_Halo_B2'
minfrm = 200
filePath = os.path.join(working_dir, working_file)

In [35]:
GB_221_B2_tracks = flikaIO.json_to_pandas(filePath, experiment_name, minfrm)
GB_221_B2_tracks['ID'] = GB_221_B2_tracks['ID'].astype(int)

In [36]:
print(GB_221_B2_tracks.loc[(GB_221_B2_tracks['ID'] == 0) & (GB_221_B2_tracks['Frame'] <= 2)][['X', 'Y']])

X           Y
0  373.713745  428.601665
1  375.241176  429.982194
2  375.226112  429.370042


In [37]:
Rg_lastFrame = fcalc.radGyr_lastFrame(GB_221_B2_tracks)

In [38]:
print(Rg_lastFrame)

[1.850764194025114, 0.948756343065203, 1.152734379680891, 0.7983096913416594, 0.9867142432575564, 2.614974112780085, 4.5375653315979285, 0.5951561965579104, 1.108494352424986, 1.2623008378681482, 0.9738460358336102, 2.4903527824421605, 1.130223893707005, 1.3169331422111394, 0.579229322774166, 1.0791144604403613, 0.9909885216148384, 1.5477657058148049, 7.361915665391719, 1.0180189979344472, 0.4271685499166152, 2.70315447789876, 0.5808880870449582, 0.8244666201143787, 1.0978312828640242, 1.033140401738235, 2.5225874217952367, 1.2280233488124512, 0.8851292531128685, 1.1292241885428629, 0.8260677184631098, 0.8959501635070519, 1.3896569609865452, 4.30879691850076, 0.9014288043947815, 0.7255743215889849, 1.856247187866016, 1.1441632210654546, 1.9747312718260213, 0.7762535603956897, 0.6611489054843899, 1.1970296929794575, 1.166734492101586, 1.0822204362693157, 2.705566049945356, 1.6665751592713953, 0.8010289653339436, 1.1453326729164808, 1.0535970456407497, 1.0664509735504808, 1.0934482333483

In [39]:
print(GB_221_B2_tracks)

Frame           X           Y   ID        Exp_Name
0         0.0  373.713745  428.601665    0  GB_221_Halo_B2
1         1.0  375.241176  429.982194    0  GB_221_Halo_B2
2         2.0  375.226112  429.370042    0  GB_221_Halo_B2
3         3.0  374.904705  428.163372    0  GB_221_Halo_B2
4         4.0  375.047256  428.657556    0  GB_221_Halo_B2
...       ...         ...         ...  ...             ...
109270  198.0  666.196771  350.592431  260  GB_221_Halo_B2
109271  199.0  665.979923  350.729597  260  GB_221_Halo_B2
109272  200.0  666.471521  351.024190  260  GB_221_Halo_B2
109273  201.0  666.565636  349.887179  260  GB_221_Halo_B2
109274  202.0  667.739407  349.951872  260  GB_221_Halo_B2

[109275 rows x 5 columns]


In [49]:
GB_221_B2_tracks['Rg'] = np.nan
for trackID in np.arange(0, len(Rg_lastFrame), 1):
    GB_221_B2_tracks.loc[GB_221_B2_tracks.ID == trackID, 'Rg'] = Rg_lastFrame[trackID]
print(GB_221_B2_tracks)

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
1195   NaN
1196   NaN
1197   NaN
1198   NaN
1199   NaN
Name: Rg, Length: 1200, dtype: float64
